In [1]:
!pip install prophet
!pip install joblib
!pip install torch
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/14.4 MB ? eta -:--:--

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 132.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 94.6 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from dateutil.relativedelta import relativedelta
import os
import glob
import boto3
import s3fs
import tarfile
import joblib
import tempfile
from config_module import Config
config = Config()

# **IMPORTANT**: Have to do this line *before* importing tensorflow
os.environ['PYTHONHASHSEED'] = str(2)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

from pickle import dump, load
import warnings


import random

# Set seed for reproducibility
np.random.seed(42)
pd.options.mode.chained_assignment = None

# Import modules
import itertools
import logging
import math
import pickle
from datetime import datetime

import ast

from prophet import Prophet
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import (mean_absolute_error, mean_absolute_percentage_error,
                             mean_squared_error)
warnings.filterwarnings('ignore')
random.seed(12)

In [3]:
def read_file(file_path):
    file_extension = os.path.splitext(file_path)[1].lower()
    
    if file_extension == '.xlsx':
        # Read Excel file
        data = pd.read_excel(file_path)
    elif file_extension == '.csv':
        # Read CSV file
        data = pd.read_csv(file_path)
    elif file_extension == '.txt':
        # Read TXT file
        data = pd.read_csv(file_path)
    else:
        raise ValueError("Unsupported file format: {}".format(file_extension))
    
    return data

In [4]:
def get_latest_file(folder_path, file_prefix):
    # Construct the search pattern for the CSV files with the given prefix
    search_pattern = os.path.join(folder_path, f"{file_prefix}*")
    
    
    # Get the list of matching files
    csv_files = glob.glob(search_pattern)
    
    
    if not csv_files:
        print("No CSV files found.")
        return None
    
    # Extract the datetime part and convert it to a datetime object
    csv_files_with_dates = []
    for file in csv_files:
        try:
            # Extract the datetime stamp from the file name
            datetime_stamp = file[len(folder_path) + len(file_prefix) + 1:-5]
            file_datetime = datetime.strptime(datetime_stamp, "%y%m%d_%H%M%S")
            csv_files_with_dates.append((file, file_datetime))
        except ValueError:
            print(f"Could not parse date from file name: {file}")
    
    if not csv_files_with_dates:
        print("No valid CSV files found.")
        return None
    
    # Sort the files by datetime in descending order and get the latest one
    latest_file = max(csv_files_with_dates, key=lambda x: x[1])[0]
    
    return latest_file

In [5]:
def get_latest_s3_file(bucket_name, prefix, file_prefix):
    s3 = boto3.client('s3')
    
    # List all objects under the given prefix
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    
    if 'Contents' not in response:
        print("No files found.")
        return None
#     print(response['Contents'])
    # Filter objects that start with the specific file prefix
    filtered_files = [
        obj for obj in response['Contents'] 
        if obj['Key'].startswith(f"{prefix}{file_prefix}")
    ]
    
    if not filtered_files:
        print("No files found with the specific prefix.")
        return None
    
    # Extract the date from each file name and store along with the key
    files_with_dates = []
    for obj in filtered_files:
        key = obj['Key']
        # Assuming the date is at the end of the filename after the last underscore
        try:
            # Get the base name without directory path
            base_name = os.path.basename(key)
            name_without_extension = os.path.splitext(base_name)[0]
            date_str = name_without_extension.split('_')[-1]  # Get the date string part
            file_date = datetime.strptime(date_str, "%Y-%m-%d")  # Parse date
            files_with_dates.append((key, file_date))
        except ValueError as e:
            print(f"Error parsing date from file name {key}: {e}")

    if not files_with_dates:
        print("No valid files with dates found.")
        return None

    # Find the file with the latest date
    latest_file_key, latest_date = max(files_with_dates, key=lambda x: x[1])
    
    return latest_file_key

In [6]:
fs = s3fs.S3FileSystem()
bucket = config.get_value('bucket')
bucket_new = config.get_value('bucket_new')
prefix_output_file = config.get_value('prefix_cleaned_training_file_path')
file_name_prefix = config.get_value('cleaned_training_filename_prefix')
file_path = get_latest_s3_file(bucket,prefix_output_file,file_name_prefix)

old_data = pd.read_excel('s3://'+f'{bucket}/{file_path}')
parts_list = old_data['part_number'].unique().tolist()

prefix_inp = config.get_value('prefix_inp')
last_tr_date = (pd.to_datetime(old_data['period']).max().strftime('%Y-%m-%d'))

In [7]:
def calculate_mape(x, y):
    """
    Function to calculate the Mean Absolute Percentage Error (MAPE) between two values x and y.
    
    Parameters:
    x: The true value.
    y: The predicted value.
    
    Returns:
    The MAPE value as a float.
    """
    
    if math.isnan(x):
        return 0
    
    # Check if x and y are equal
    if x == y:
        return 1
    
    # Check if x is zero
    elif x == 0:
        return 0
    
    # Calculate MAPE
    else:
        mape = 1 - (abs(x - y) / x)
        return mape

In [8]:
def prophet_forecast(parts_data_period, parts_data_train, parts_data_test, param_dict, period_lag):
    """
    Perform a forecast using Prophet for multiple hyperparameter combinations.
 
    Args:
    - data (DataFrame): Input DataFrame containing actual values for multiple products.
    - hyperparameters (list of dict): List of dictionaries containing hyperparameter combinations.
 
    Returns:
    - DataFrame: DataFrame containing actuals, fitted values, and future predictions for each product and hyperparameter combination.
    """
    results = []

    # Initialize Prophet model with hyperparameters
    model = Prophet(**param_dict)

    model.fit(parts_data_train.rename(columns={'period': 'ds', 'Total_qty_per_month': 'y'}))
    # Make future dataframe
    future = model.make_future_dataframe(periods=period_lag, freq='MS')
    forecast = model.predict(future)


    sample_result = forecast[['ds', 'yhat']].rename(columns={'ds': 'period', 'yhat': 'fitted'})
    sample_result['part_number'] = parts_data_period['part_number'].iloc[0]
    sample_result['actual'] = parts_data_period['Total_qty_per_month'].values
    sample_result['type'] = None
    sample_result['type'].iloc[:-period_lag] = 'Train'
    sample_result['type'].iloc[-period_lag:] = 'Test'
    

    sample_result['model_name'] = str(param_dict)
    results.append(sample_result)
    return pd.concat(results, ignore_index=True)

In [9]:
def create_test_complete_dataset_from_scratch(df, start_date, end_date, pn):
    """
    Function to create a complete dataset from scratch for a given part number (pn) over a specified date range.
    
    Parameters:
    df (DataFrame): The input DataFrame (not used in this function but kept for consistency).
    start_date (datetime): The start date for the period.
    end_date (datetime): The end date for the period.
    pn (str): The part number for which the dataset is being created.
    
    Returns:
    DataFrame: A DataFrame with all months in the specified period and the specified part number.
    """
    
    start_date = start_date.strftime('%Y-%m-%d')
    end_date = end_date.strftime('%Y-%m-%d')
    
    start_date = datetime.strptime(start_date, '%Y-%m-%d')
    end_date = datetime.strptime(end_date, '%Y-%m-%d')
     
    date_range = pd.date_range(start=start_date, end=end_date, freq='MS')
    all_months_df = pd.DataFrame(date_range, columns=['period'])
    all_months_df['period'] = all_months_df['period'].dt.strftime('%Y-%m')
    
    all_product_df = pd.DataFrame({'part_number': [pn]})
    
    #Cross-merging date DataFrame and part number DataFrame
    merged_df = all_months_df.merge(all_product_df, on=None, how='cross')
    merged_df['period'] = pd.to_datetime(merged_df['period'])
    
    return merged_df

def create_test_complete_dataset(df, start_date, end_date):
    """
    Function to create a complete dataset with all months in the specified period for each part number in the input DataFrame.
    
    Parameters:
    df (DataFrame): The input DataFrame containing at least 'part_number' and 'period' columns.
    start_date (datetime): The start date for the period.
    end_date (datetime): The end date for the period.
    
    Returns:
    DataFrame: A DataFrame with all months in the specified period for each part number in the input DataFrame.
    """
    
    start_date = start_date.strftime('%Y-%m-%d')
    end_date = end_date.strftime('%Y-%m-%d')
    
    start_date = datetime.strptime(start_date, '%Y-%m-%d')
    end_date = datetime.strptime(end_date, '%Y-%m-%d')
    
    date_range = pd.date_range(start=start_date, end=end_date, freq='MS')
    all_months_df = pd.DataFrame(date_range, columns=['period'])
    all_months_df['period'] = all_months_df['period'].dt.strftime('%Y-%m')
    
    all_product_df = pd.DataFrame({'part_number': df['part_number'].unique()})
    
    merged_df = all_months_df.merge(all_product_df, on=None, how='cross')
    merged_df['period'] = pd.to_datetime(merged_df['period'])
    
    merged_df = merged_df.merge(df, left_on=['period', 'part_number'], right_on=['period', 'part_number'], how='left')
    
    return merged_df

In [10]:
def create_complete_dataset(data_new, train_start_date, train_end_date):
    """
    Function to create training and testing datasets from the input data for the specified training period.
    
    Parameters:
    data_new (DataFrame): The input DataFrame containing at least 'part_number' and 'period' columns.
    train_start_date (str): The start date for the training period in 'YYYY-MM-DD' format.
    train_end_date (str): The end date for the training period in 'YYYY-MM-DD' format.
    
    Returns:
    tuple: A tuple containing two DataFrames: the training dataset and the testing dataset.
    """
    
    data_new['period'] = pd.to_datetime(data_new['period']).dt.strftime('%Y-%m-%d')
    
    data_new = data_new.sort_values(['part_number', 'period'])
    
    data_new = data_new[data_new['period'] >= train_start_date]
    
    data_new['period'] = pd.to_datetime(data_new['period'])
    
    df_filtered_1 = data_new.copy()
    
    df_train = df_filtered_1[df_filtered_1['period'] <= train_end_date]
    
    test_start_date = (pd.to_datetime(train_end_date) + pd.DateOffset(months=1)).strftime('%Y-%m-%d')
    test_end_date = (pd.to_datetime(train_end_date) + pd.DateOffset(months=3)).strftime('%Y-%m-%d')
    
    df_test = df_filtered_1[(df_filtered_1['period'] >= test_start_date) & (df_filtered_1['period'] <= test_end_date)]
    return df_train, df_test

In [11]:
def create_TS_model_prophet(df, hyp_dict, p, t_start_date, t_end_date, lag_data):
    """
    Function to create a time series model using Prophet, including handling for train-test split and missing test data.
    
    Parameters:
    df (DataFrame): The input DataFrame containing at least 'part_number' and 'period' columns.
    hyp_dict (dict): Dictionary containing hyperparameters for the Prophet model.
    p: Additional parameters required by the Prophet model.
    t_start_date (str): The start date for the training period in 'YYYY-MM-DD' format.
    t_end_date (str): The end date for the training period in 'YYYY-MM-DD' format.
    lag_data: Additional lag data required by the model.
    
    Returns:
    DataFrame: A DataFrame containing the Prophet model forecasts.
    """
    
    pn = df['part_number'].unique().tolist()[0]
    
    df_new = df.copy()
    
    # Splitting the data into training and testing sets
    train, test = create_complete_dataset(df_new, t_start_date, t_end_date)
    train_clean = train.copy()
    
    if test.empty:
        test = create_test_complete_dataset_from_scratch(test, 
                                                         train_clean['period'].max() + pd.DateOffset(months=1), 
                                                         train_clean['period'].max() + pd.DateOffset(months=3),
                                                         pn)
    
    if test.shape[0] < 3:
        test = create_test_complete_dataset(test, 
                                            train_clean['period'].max() + pd.DateOffset(months=1), 
                                            train_clean['period'].max() + pd.DateOffset(months=3))
    
    data_start_date = train['period'].min()
    train_end_date = train['period'].max()
    test_start_date = test['period'].min()
    data_end_date = test['period'].max()
    
    # Combining training and testing data for the period data
    period_data = pd.concat([train_clean, test], ignore_index=True)
    
    prophet_all_df = prophet_forecast(period_data, train_clean, test, hyp_dict, p)
    return prophet_all_df

In [12]:
def check_input_type(input_value):
    """
    Function to check the type of the input value and convert it to a string if it is an integer.
    
    Parameters:
    input_value: The value to be checked and possibly converted.
    
    Returns:
    The input value converted to a string if it is an integer, otherwise the original input value.
    """
    
    # Checking the type of the input value
    if isinstance(input_value, int):
        # Input value is of type int: {input_value}. Converting to string.
        return str(input_value)
    else:
        return input_value

In [13]:
def extract_metrics(parts_df_seg, data_set, train_start_date, train_end_date, lag_val, prophet_config):
    """
    Function to extract metrics using time series modeling (Prophet) for each part number in the input DataFrame segment.
    
    Parameters:
    parts_df_seg (list): List of part numbers to process.
    data_set (DataFrame): The input dataset containing at least 'part_number' and 'period' columns.
    train_start_date (str): The start date for the training period in 'YYYY-MM-DD' format.
    train_end_date (str): The end date for the training period in 'YYYY-MM-DD' format.
    lag_val: Additional lag value required for modeling.
    
    Returns:
    DataFrame: A DataFrame containing metrics extracted using time series modeling for each part number.
    """
    
    final_best_df_data = pd.DataFrame()
    prophet_data_full = pd.DataFrame()
    index_val = 0
    
    
    for parts in parts_df_seg:
        try:
            print(f"Processing part number: {parts}")
            parts = check_input_type(parts)

            parts_data = data_set[data_set['part_number'] == parts]
            param_det = prophet_config[prophet_config['part_number']==parts]['model_name'].values[0]
            params_prophet = ast.literal_eval(param_det)
    
            prophet_data = create_TS_model_prophet(parts_data, params_prophet, 3, train_start_date, train_end_date, lag_val)
            prophet_data_full = pd.concat([prophet_data_full, prophet_data], ignore_index=True)
            pivot_data_prophet = prophet_data.pivot(index='period', columns='model_name', values='fitted')
            
            merged_data_prophet = prophet_data.drop(['fitted', 'model_name'], axis=1).drop_duplicates().merge(pivot_data_prophet, 
                                                                                                              how='left', on='period')
            final_best_df_data = pd.concat([final_best_df_data, merged_data_prophet], 
                                           ignore_index=True)
            
            index_val += 1
        except Exception as e:
            print(f"Error processing part number {parts}: {e}")
    
    # Completed extracting metrics for all parts
    print("Completed extracting metrics for all parts")
    return prophet_data_full

In [14]:
def create_bestfit_prophet_forecast(list_parts_all, 
                                    main_data, 
                                    start_date_train, 
                                    end_date_train, 
                                    lag_val, 
                                    prophet_model_config_df):
    """
    Function to create the best fit Prophet model for training data and output the results to an Excel file.

    Parameters:
    list_parts_all (list): List of part numbers to process.
    main_data (DataFrame): Main dataset containing all necessary columns including 'part_number' and 'period'.
    start_date_train (str): Start date of the training period in 'YYYY-MM-DD' format.
    end_date_train (str): End date of the training period in 'YYYY-MM-DD' format.
    lag_val (int): Lag value used for modeling.
    output_path_folder (str): Folder path where the output Excel file will be saved.
    prophet_bestfit_filename (str): file name to store bestfit
    all_prophet_fitted_forecast_filename (str): file name to store all prophet_results

    Returns:
    None
    """
    
    # Extract metrics for all parts
    parts_200_df = extract_metrics(list_parts_all, main_data, start_date_train, end_date_train, lag_val, prophet_model_config_df)

    
    return parts_200_df

In [15]:
prefix_bestfit_file=config.get_value('prefix_bestfit_file')
prefix_bestfit_file=prefix_bestfit_file+'/'
file_name_bestfit =config.get_value('file_name_bestfit')
latest_best_fit_data_url = get_latest_s3_file(bucket_new,prefix_bestfit_file, file_name_bestfit)
print(latest_best_fit_data_url)
latest_best_fit_data = pd.read_excel('s3://'+f'{bucket_new}/{latest_best_fit_data_url}')


# prefix_bestfit_file=config.get_value('prefix_bestfit_file')
# prefix_bestfit_file=prefix_bestfit_file+'/'
# file_name_bestfit =config.get_value('file_name_bestfit')
# latest_best_fit_data_url = get_latest_s3_file(bucket_new,prefix_bestfit_file, file_name_bestfit)
# print(latest_best_fit_data_url)
# latest_best_fit_data = pd.read_excel('s3://'+f'{bucket_new}/{latest_best_fit_data_url}')

curated-bo/curated-bo-demand-forecast/bestfit-selected-models/bestfit_selected_models_historic_months_2024-11-03.xlsx


In [16]:
print(latest_best_fit_data_url)

curated-bo/curated-bo-demand-forecast/bestfit-selected-models/bestfit_selected_models_historic_months_2024-11-03.xlsx


In [17]:
pd.to_datetime(config.get_value('model_training_date'),format="%Y%m%d").strftime('%Y-%m-%d')

'2024-09-13'

In [18]:
prophet_config_data_prefix = f"{config.get_value('primary_path')}/{config.get_value('tpcap_model')}/{config.get_value('model_training_date')}/prophet"
prophet_config_data_prefix_1 = prophet_config_data_prefix + '/'
prophet_config_filename = config.get_value('prophet_config_filename')
# print(f"s3://{bucket}/{prophet_config_data_prefix_1}{prophet_config_filename}{pd.to_datetime(config.get_value('model_training_date'),format='%Y%m%d').strftime('%Y-%m-%d')}.xlsx")
model_config_dataset_path = get_latest_s3_file(bucket,prophet_config_data_prefix_1,prophet_config_filename)
print(model_config_dataset_path)
model_config_dataset = pd.read_excel('s3://'+f'{bucket}/{model_config_dataset_path}')
# model_config_dataset = pd.read_excel(f"s3://{bucket}/{prophet_config_data_prefix_1}{prophet_config_filename}{pd.to_datetime(config.get_value('model_training_date'),format='%Y%m%d').strftime('%Y-%m-%d')}.xlsx")

persist-bo/persist-bo-demand-forecast/tpcap-models/20240913/prophet/train_prophet_config_2024-09-13.xlsx


In [19]:
# model_config_dataset

In [20]:
print(datetime.now())

2024-11-05 11:17:28.938093


In [21]:
prophet_parts = latest_best_fit_data[latest_best_fit_data['model_name']=="Prophet"]['part_number'].unique().tolist()
start_training_date=old_data['period'].min().strftime('%Y-%m-%d')
latest_date = old_data['period'].max()
last_training_date = latest_date.strftime('%Y-%m-%d')
parts_list_updated = prophet_parts[:]
print(len(parts_list_updated))
final_forecast_prophet = create_bestfit_prophet_forecast(parts_list_updated, old_data,start_training_date,last_training_date , 0,  
                             model_config_dataset)


11:17:29 - cmdstanpy - INFO - Chain [1] start processing


11:17:29 - cmdstanpy - INFO - Chain [1] done processing


3741
Processing part number: 04000081B0


11:17:29 - cmdstanpy - INFO - Chain [1] start processing


11:17:29 - cmdstanpy - INFO - Chain [1] done processing


...

11:31:01 - cmdstanpy - INFO - Chain [1] done processing


Processing part number: PZ0570K05500
Processing part number: PZT0038023


11:31:01 - cmdstanpy - INFO - Chain [1] start processing


11:31:01 - cmdstanpy - INFO - Chain [1] done processing


Completed extracting metrics for all parts


In [22]:
print(datetime.now())

2024-11-05 11:31:01.485191


In [23]:
prefix_prophet_forecast = config.get_value('prefix_model_output_forecast_path')
full_path_prophet_forecast = f"s3://{bucket}/{prefix_prophet_forecast}"
file_date_forecasting = pd.to_datetime(config.get_value('model_forecasting_date')).strftime("%Y-%m-%d")
prophet_forecast_file_name = f'{config.get_value("prophet_forecast_file_name")}{file_date_forecasting}.xlsx'

# saving file
final_forecast_prophet.to_excel(f"{full_path_prophet_forecast}/{config.get_value('model_forecasting_date')}/{prophet_forecast_file_name}", index=False)

In [24]:
f"{full_path_prophet_forecast}/{config.get_value('model_forecasting_date')}/{prophet_forecast_file_name}"

's3://s3-dx-tdem-ie-nonprod-persist/persist-bo/persist-bo-demand-forecast/model-forecasted-output/20241103/prophet_forecast_data_2024-11-03.xlsx'

In [25]:
final_forecast_prophet['part_number'].nunique()

3741